## Importar bibliotecas

**pandas**: Para pré-processamento dos dados.  
**dbfread**: Leitura dos arquivos brutos em DBF. Precisamos importar somente a função DBF.  
**os**: Para trabalhar com os diretórios dos arquivos.

In [1]:
import pandas as pd
from dbfread import DBF
import os

## Carregar multiplos arquivos

### Apontar diretórios, colunas desejadas e lista vazia
1. Primeiro vamos definir os diretórios onde os dados brutos estão. Na pasta foram colocados somente os arquivos de 2014 a 2023 para iniciar teste com base em 10 anos. Há dois caminhos para PCs diferentes, pois estamos trabalhando colaborativamente em duas máquinas.
2. Atribuímos os nomes das colunas de interesse a um objeto 'colunas_selecionadas'.
3. Criamos um objeto lista vazia, que receberá os dataframes, um momento antes de serem concatenados.

In [ ]:
# 1. Diretorio contendo os arquivos .dbf. 

# Caminho PC Costela
# diretorio_dbf = "/home/costela/Documentos/Escorpiao-series/Dados_brutos/"

# Caminho PC Usuario
diretorio_dbf = "/home/usuario/Documentos/Escorpiao_series/Dados_brutos/"


# 2. Selecao das colunas de interesse
colunas_selecionadas = ['DT_SIN_PRI', 'SEM_PRI','ANO_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 
        'CS_RACA', 'CS_ESCOL_N','ID_MN_RESI', 'ID_OCUPA_N', 'ANT_DT_ACI', 'ANT_UF', 'ANT_MUNIC_', 
        'SG_UF', 
        #'ANT_LOCALI', # coluna removida por nao existir em todos os anos
        'ANT_TEMPO_','ANT_LOCA_1', 'MCLI_LOCAL', 'CLI_DOR', 'CLI_EDEMA', 
        'CLI_EQUIMO','CLI_NECROS', 'CLI_LOCAL_', 'CLI_LOCA_1', 'MCLI_SIST', 'CLI_NEURO',
        'CLI_HEMORR', 'CLI_VAGAIS', 'CLI_MIOLIT', 'CLI_RENAL', 'CLI_OUTR_2',
        'CLI_OUTR_3', 'CLI_TEMPO_', 'TP_ACIDENT', 'ANI_TIPO_1', 'ANI_SERPEN',
        'ANI_ARANHA', 'ANI_LAGART', 'TRA_CLASSI', 'CON_SOROTE', 'NU_AMPOLAS',
        'NU_AMPOL_1', 'NU_AMPOL_8', 'NU_AMPOL_6', 'NU_AMPOL_4', 'NU_AMPO_7',
        'NU_AMPO_5', 'NU_AMPOL_9', 'NU_AMPOL_3', 'COM_LOC', 'COM_SECUND',
        'COM_NECROS', 'COM_COMPOR', 'COM_DEFICT', 'COM_APUTAC', 'COM_SISTEM',
        'COM_RENAL', 'COM_EDEMA', 'COM_SEPTIC', 'COM_CHOQUE', 'DOENCA_TRA',
        'EVOLUCAO', 'DT_OBITO', 'DT_ENCERRA', 'DT_DIGITA']

# 3. Lista para armazenar os dataframes
dataframes = []

### Ler cada um dos arquivos DBF 
O loop abaixo funciona assim: ele percorre a lista de arquivos existente no diretório e, para cada item, verifica se a extensão é .dbf. Quando é, o nome do arquivo é concatenado ao caminho-base do diretório, formando o caminho completo (endereço) do arquivo.  

Esse caminho completo é então passado para a função de leitura do DBF, utilizando o encoding latin1, o que ajuda a preservar corretamente palavras com acentos e outros caracteres especiais. Cada arquivo lido é armazenado inicialmente no objeto tabela, pois arquivos DBF são carregados como tabelas/iteráveis, e não diretamente como dataframes.  

Na sequência, cada tabela é convertida para dataframe, aplica-se o filtro para manter apenas registros do estado de São Paulo e apenas acidentes escorpiônicos, e então o dataframe é reduzido às colunas previamente definidas em colunas_selecionadas. Por fim, cada dataframe resultante é adicionado a uma lista por meio da função append.

In [ ]:
# 4. Iterar sobre os arquivos no diretorio

for arquivo in os.listdir(diretorio_dbf):  # Iterar sobre os arquivos no diretorio (na pasta estao somente 2014 a 2023, os demais estão no drive)
    if arquivo.endswith(".dbf"):    # Verificar se o arquivo é um .dbf
        caminho_arquivo = os.path.join(diretorio_dbf, arquivo) # monta o caminho completo do arquivo
        tabela = DBF(filename=caminho_arquivo, load=True, encoding='latin1') # Carregar o arquivo DBF
        df = pd.DataFrame(tabela)   # Converter para DataFrame do pandas
        df = df[df['SG_UF']=='35']  # Filtrar apenas registros de SP
        df = df[df['TP_ACIDENT']=='3']  # Filtrar apenas acidentes por escorpião
        df = df[colunas_selecionadas]  # Selecionar colunas de interesse
        dataframes.append(df)   # Adicionar o DataFrame à lista

### Juntar todos os arquivos em um
Todos os arquivos contidos na lista 'df' serão concatenados para um novo objeto 'df_concatenado', que consiste então em um grande dataframe.

In [8]:
# Concatenar todos os DataFrames em um unico (2014 a 2023)
df_concatenado = pd.concat(dataframes, ignore_index=True) # Concatenar todos os DataFrames em um unico


### Transformar colunas de data em datetime e as demais em texto
Essa etapa é necessária para viabilizar a recodificação das variáveis sem inconsistências de tipo. Ao converter as colunas de data para o formato data/hora (datetime) e padronizar as demais como texto, reduzimos o risco de erros durante filtros, comparações, junções e recodificações, garantindo que o conjunto de dados fique coerente e uniforme para as próximas etapas do processamento.  

Inicialmente, define-se a lista col_datas com os nomes das colunas que devem ser tratadas como datas. Em seguida, é obtida a interseção entre essa lista e as colunas realmente presentes no dataframe (cols_d), o que evita erros caso alguma coluna não exista no conjunto de dados.

As colunas identificadas em cols_d são então convertidas para o tipo datetime usando pd.to_datetime, com errors='coerce' (valores inválidos passam a NaT) e dayfirst=True (interpretando datas no padrão brasileiro, dia/mês/ano).

Por fim, todas as demais colunas — isto é, aquelas que não pertencem a cols_d — são convertidas para o tipo texto (string). Com isso, o dataframe fica padronizado: colunas de data em formato apropriado para análises temporais e o restante em texto, reduzindo inconsistências de tipo e facilitando etapas posteriores de recodificação e tratamento das variáveis.

In [16]:
col_datas = ['DT_SIN_PRI','DT_NOTIFIC','DT_NASC','DT_OBITO','DT_ENCERRA','DT_DIGITA','ANT_DT_ACI']
cols_d = df_concatenado.columns.intersection(col_datas)
df_concatenado[cols_d] = df_concatenado[cols_d].apply(pd.to_datetime, errors='coerce', dayfirst=True)

df_concatenado[df_concatenado.columns.difference(cols_d)] = df_concatenado[df_concatenado.columns.difference(cols_d)].astype("string")

## Salvar banco
O banco de dados foi salvo em formato Parquet para preservar os tipos das variáveis (datas, textos e numéricos) e evitar ambiguidades comuns em formatos como CSV ou XLSX, nos quais códigos podem ser interpretados ora como texto, ora como número na reabertura do arquivo. Essa escolha reduz o risco de inconsistências na etapa seguinte de recodificação e facilita a reprodutibilidade do processamento.

Além disso, o arquivo é gravado em dois caminhos alternativos, pois o trabalho está sendo realizado em duas máquinas. Assim, basta habilitar a linha correspondente ao computador em uso.


In [18]:
# Salvar o DataFrame concatenado em um arquivo .parquet

# PC Costela
# df_concatenado.to_parquet("/home/costela/Documentos/Escorpiao-series/Dados_processados/df_concatenado.parquet")

# PC Usuario
df_concatenado.to_parquet("/home/usuario/Documentos/Escorpiao_series/Dados_processados/df_concatenado.parquet")
